In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rrmac\\OneDrive\\Desktop\\DePaul_classes\\SE-489\\project\\test5\\group_project_se489\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rrmac\\OneDrive\\Desktop\\DePaul_classes\\SE-489\\project\\test5\\group_project_se489'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class GettingDataConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from se489_group_project.constants import *
from se489_group_project.utility.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.data_storage])

    def get_data_ingestion_config(self) -> GettingDataConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = GettingDataConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import zipfile
import gdown
from se489_group_project import logger
from se489_group_project.utility.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: GettingDataConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("data/raw", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-07 21:50:02,894: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-07 21:50:02,903: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-07 21:50:02,904: INFO: common: created directory at: data]
[2024-05-07 21:50:02,905: INFO: common: created directory at: data/raw]
[2024-05-07 21:50:02,906: INFO: 3725992822: Downloading data from https://drive.google.com/file/d/1gesLApompvvnzz-AWyWM4ikmk7BOGWAp/view?usp=sharing into file data/raw/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1gesLApompvvnzz-AWyWM4ikmk7BOGWAp
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1gesLApompvvnzz-AWyWM4ikmk7BOGWAp&confirm=t&uuid=7034491a-70b5-4162-82b0-6c919d764f0a
To: c:\Users\rrmac\OneDrive\Desktop\DePaul_classes\SE-489\project\test5\group_project_se489\data\raw\data.zip
100%|██████████| 57.7M/57.7M [00:02<00:00, 19.6MB/s]

[2024-05-07 21:50:07,318: INFO: 3725992822: Downloaded data from https://drive.google.com/file/d/1gesLApompvvnzz-AWyWM4ikmk7BOGWAp/view?usp=sharing into file data/raw/data.zip]
